# Description

This tutorial aims to help you understand how to run the stacking code and to produce the necessary configuration files. This tutorial is prepared in the form of a jupyter notebook and should be used from a runnable interface (e.g. Jupyter Notebook Server, Virtual Studio Code, ...). The html view of this tutorial might provide incomplete and/or out of date information.

# Running stacking

In order to compute the composite spectra you need to run the code run_stacking. In this tutorial we are going to see how. If you are running this tutorial, you should have the package stacking installed. If you do not have it, then check the INSTALLATION section of the README

The usage of the code is very simple. You just need to run from the command line 

`python run_stacking.py config.ini`

where `config.ini` is a file with the desired configuration. 

Alternatively, you might want to run it from this jupyter notebook. In that case we do it like this:




In [3]:
# import main function from stacking
import sys
import os
sys.path.append(os.path.expanduser('/path-to-stacking/stacking/bin/')) # this should be changed with your path to stacking
from run_stacking import main

In [ ]:
# run it using a configuration file
import argparse
parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description="Compute the stack of a set of spectra")

parser.add_argument(
'config_file',
type=str,
default=None,
help=('Configuration file. To learn about all the available options '
        'check the configuration tutorial in '
        'tutorials/configuration_tutorial.ipynb'))

print("WARNING: depending on the configuration file used this might take a lot of time")
main(parser.parse_args())

Some questions you might have:
- What if I want to run without saving the configuration in a file? Unfortunatly this is not possible, at least for now. A different initialisation of the Config class would need to be coded.
- Can I check the configuration of a previous run? Yes, the configuration for a given run is stored at a file named .config.ini in the output folder. 
- Can I use enviroment variables in my config arguments? Yes, they will be expanded before calling the specified file.
- How can I create my own config? This is where things start to get tricky. We are now going to review how to build a configuration file, and the different options that are available.

# How to build a config file: General structure

The configuration file is nothing but a bunch of arguments that are structured in different sections. They are then parsed into usable format. To create a section simply type its name inside of square braquets:

`[new section]`

To add an argument to this section specify it simply:

`my_string_argument = "a string"`

`also_a_string_argument = another string`

`my_int_argument = 1`

`my_float_argument = 1.4`

`my_bool_argument = True`

`also_a_bool_argument = yes`
`

Note that the arguments and sections are not ordered, but all the arguments in a given section will need to be placed before the next section begins.

Now that we have seen a short general introduction on config files, let's move to the more specific task at hand: how to build a config file for `picca.delta_extraction`. In this configuration we need some sections to be present and each section will require some specific parameters. Depending on the options, other sections might be required/used, but we'll come back to this later. These are the mandatory sections:
- `[general]`
- `[reader]`
- `[rebin]`
- `[normalizer]`
- `[stacker]`
- `[writer]`

We now revise the arguments of the different sections. Note that if other arguments are passed, they will be ignored.

## General section

The general section sets common options that are not really used by any of the classes specifically. You can get the up-to-date list of arguments for this section by running the following cell

In [1]:
from stacking.configuration_help import print_general_options
print_general_options()

`overwrite`: This variable specifies where to save the results. **Type: str**, **Required: no**, **Default: False**

`logging level console`: This variable controls console messages. Must be one of CRITICAL, ERROR, WARNING_OK, WARNING, INFO, PROGRESS, DEBUG, NOTSET **Type: str**, **Required: no**, **Default: PROGRESS**

`logging level file`: This variable controls log messages. Must be one of CRITICAL, ERROR, WARNING_OK, WARNING, INFO, PROGRESS, DEBUG, NOTSET. Ignored if `log` is `None` **Type: str**, **Required: no**, **Default: PROGRESS**

`log`: If a log file is passed, print messages also there **Type: str**, **Required: no**, **Default: run.log**

`output directory`: This variable specifies where to save the results. **Type: str**, **Required: yes**

`num processors`: Number of processors to be used for multiprocessed tasks (e.g. data i/o, expected flux). 0 for using half the processes available on the machine (subprocess will take its default value). **Type: int or None**, **Required: no**, **Default: 0**

`run type`: Run type. Must be one of 'normal' 'merge norm factors' 'merge stack'. **Type: str**, **Required: no**, **Default: normal**

## Reader section
The reader section controls which type of data are we going to load. For example, we might want to run stacking on SDSS spectra or on DESI data. You can get the current list of available readers by running the following cell


In [1]:
from stacking.configuration_help import print_classes
print_classes("readers")

Dr16Reader


To get the class description run the cell below

In [1]:
from stacking.configuration_help import print_class_description
print_class_description("Dr16Reader", "readers") # this should be changed with your reader class

### Class Dr16Reader

Reads the spectra from SDSS DR16 and formats its data as a list of
    Spectrum instances.

    Methods
    -------
    (see Reader in stacking/reader.py)
    __init__
    __parse_config
    read_data
    read_drq_catalogue
    read_from_spec
    read_from_spplate
    read_spall
    trim_catalogue

    Attributes
    ----------
    (see Reader in stacking/data.py)

    best_obs: bool
    If True, reads only the best observation for quasars with repeated
    observations

    max_balnicity_index: float or None
    Maximum value allowed for the Balnicity Index to keep the quasar.
    None for no maximum

    max_num_spec: int or None
    Maximum number of spectra to load. None for no maximum.
    Multiple spectra comming from the same line of sight are not counted here

    drq_filename: str
    Filename of the DRQ catalogue

    keep_bal: bool
    If False, remove the quasars flagged as having a Broad Absorption
    Line. Ignored if max_balnicity_index is not None

    logger: logging.L

### Class Reader

Abstract class to define the readers skeleton

    Methods
    -------
    __init__
    __parse_config

    Attributes
    ----------
    catalogue: astropy.table.Table
    Metadata associated with the spectra. Ordering should be maintained
    between spectra and catalogue

    input_directory: str
    The input directory

    spectra: list of Spectrum
    The read spectra. Ordering should be maintained
    between spectra and catalogue
    


You can now access the options of a given class by executing the following cell

In [1]:
from stacking.configuration_help import print_class_options
print_class_options("Dr16Reader", "readers") # this should be changed with your reader class

### Class Dr16Reader

#### Options:

`input directory`: Directory containing the spectra. **Type: str**, **Required: yes**

`best obs`: If True, reads only the best observation for quasars with repeated observations. **Type: bool**, **Required: no**, **Default: False**

`drq catalogue`: Filename of the DRQ catalogue. **Type: str**, **Required: yes**

...

`read mode`: Reading mode. Currently supported reading modes are 'spplate' and 'spec'. **Type: str**, **Required: no**, **Default: spplate**

`skip N first spec`: Skip the N first spectra in the catalogue. **Type: int**, **Required: no**, **Default: 0**

`spAll`: Path to the spAll file required for multiple observations. **Type: str**, **Required: no**

`z min`: Minimum redshift. Quasars with redshifts lower than z_min will be discarded. **Type: float**, **Required: no**, **Default: 0.0**

`z max`: Maximum redshift. Quasars with redshifts higher than or equal to z_max will be discarded. **Type: float**, **Required: no**, **Default: 10.0**

## Rebin section
The rebin section controls the parameters for the common wavelength grid to where we rebin all objects before actually stacking them. This is managed by the class `Rebin` whose up-to-date options can be listed running the cell below

In [3]:
from stacking.configuration_help import print_class_options
print_class_options("Rebin", ".")

### Class Rebin

#### Options:

`max wavelength`: Maximum wavelength of the common wavelength grid. **Type: float**, **Required: yes**

`min wavelength`: Minimum wavelength of the common wavelength grid. **Type: float**, **Required: yes**

`step type`: Type of step in the common grid. 'lin' means the common grid is equally spaced in wavelength. 'log' means it is equally spaced in the logarithm of the wavelength. **Type: string**, **Required: yes**

`step wavelength`: Step in the common grid. **Type: float**, **Required: yes**

## Normalizer section
The normalizer section controls the parameters for the normalization procedure. Different data may use different normalization procedures. You can get the current list of available normalizers by running the following cell

In [2]:
from stacking.configuration_help import print_classes
print_classes("normalizers")

PartialMultipleRegionsNormalization
MergeMultipleRegionsNormalization
NoNormalization
MultipleRegionsNormalization


To get the class description run the cell below

In [2]:
from stacking.configuration_help import print_class_description
print_class_description("MergeMultipleRegionsNormalization", "normalizers") # this should be changed with your normalizer class

### Class MergeMultipleRegionsNormalization

This class is set to compute the normalization factors using multiple
    normalization regions.

    Contrary to the parent class, here we assume we have a set of partial runs.
    They are merged and the correction factors are computed.
    Thus, after computing the normalization factors, they will be saved and the
    program will end.

    To be implemented:
    Optionally combine with normalization factors loaded from previous runs

    Methods
    -------
    (see MultipleRegionsNormalization in stacking/normalizers/multiple_regions_normalization.py)
    __init__
    __parse_config
    compute_norm_factors

    Attributes
    ----------
    (see MultipleRegionsNormalization in stacking/normalizers/multiple_regions_normalization.py)

    folders_list: list of str
    List of folders where the partial runs are stored

    logger: logging.Logger
    Logger object

    save_on_list: bool
    If True, save the normalization factors on the list of folders
    


### Class MultipleRegionsNormalization

This class is set to compute the normalization factors using multiple
    normalization regions

    Methods
    -------
    __init__
    __parse_config
    compute_normalisation_factors
    normalize_spectrum

    Attributes
    ----------
    correction_factors: array of float
    Correction factors that relate the different intervals

    intervals:  array of (float, float)
    Array containing the selected intervals. Each item must contain
    two floats signaling the starting and ending wavelength of the interval.
    Naturally, the starting wavelength must be smaller than the ending wavelength.

    log_directory: str
    Directory where log data is saved. Normalization factors will be saved there

    logger: logging.Logger
    Logger object

    main_interval: int
    Number of main normalizeation interval

    norm_factor: pd.DataFrame
    Pandas DataFrame with the normalization factors

    num_intervals: int
    Number of intervals

    save_format: str
    Saving format, e.

### Class Normalizer

Abstract class to define the normalizer skeleton

    Methods
    -------
    compute_normalisation_factors
    normalize_spectrum
    


You can now access the options of a given class by executing the following cell

In [1]:
from stacking.configuration_help import print_class_options
print_class_options("PartialMultipleRegionsNormalization", "normalizers") # this should be changed with your normalizer class

### Class PartialMultipleRegionsNormalization

#### Options:

`intervals`: Normalzation intervals. Expected format is 'start0 - end0, start1 - end1, ..., startN - endN' where startX and endX are positive numbers. **Type: str**, **Required: no**, **Default: 1300 - 1500, 2000 - 2600, 4400 - 4800**

`log directory`: Directory where log data is saved. Normalization factors will be saved there. **Type: str**, **Required: yes**

`main interval`: Number of main normalization interval. **Type: int**, **Required: no**, **Default: 1**

`min nrom sn`: Minimum S/N for the normalization factor to be considered. **Type: float**, **Required: no**

`num processors`: Number of processors to use. **Type: int**, **Required: no**

`save format`: Saving format, e.g. 'csv', 'txt', 'fits' or 'fits.gz'. **Type: str**, **Required: no**, **Default: fits.gz**

`sigma_I`: Additional variance added to the inverse variance of the spectra to suppress the brightest pixels. **Type: float**, **Required: no**, **Default: 0.05**

`compute correction factors flag`: If True, compute the correction factors. **Type: bool**, **Required: no**, **Default: False**

## Stacker section
The stacker section controls the stacking parameters. Different stacking classes use different procedures: mean, median, multiple stack, ... You can get the current list of available normalizers by running the following cell 

In [3]:
from stacking.configuration_help import print_classes
print_classes("stackers")

BootstrapMeanStacker
SplitStacker
MergeSplitStacker
MergeStacker
MedianStacker
BootstrapSplitMeanStacker
MergeMedianStacker
MergeSplitMeanStacker
MergeBootstrapMeanStacker
MergeMeanStacker
MergeSplitMedianStacker
BootstrapStacker
SplitMedianStacker
SplitMeanStacker
MeanStacker


To get the class description run the cell below

In [3]:
from stacking.configuration_help import print_class_description
print_class_description("MergeSplitMedianStacker", "stackers") # this should be changed with your stacker class

### Class MergeSplitMedianStacker

Class to compute multiple stacks splitting on one
    or more properties of the spectra. Uses class MergeMedianStacker

    Methods
    -------
    (see MergeSplitStacker in stacking/stackers/merge_split_stacker.py)
    (see MergeMeanStacker in stacking/stackers/merge_median_stacker.py)

    Attributes
    ----------
    (see MergeSplitStacker in stacking/stackers/merge_split_stacker.py)
    (see MergeMeanStacker in stacking/stackers/merge_median_stacker.py)
    


### Class MergeSplitStacker

Abstract class to compute multiple stacks splitting on one
    or more properties of the spectra using different partial runs

    Methods
    -------
    (see MergeStacker in stacking/stackers/merge_stacker.py)
    __init__

    Attributes
    ----------
    (see MergeStacker in stacking/stackers/merge_stacker.py)

    groups_info: pd.DataFrame
    DataFrame containing the group information

    num_groups: int
    Number of groups the data is split on

    split_catalogue: pd.DataFrame
    The catalogue to be split
    


### Class MergeMedianStacker

...

Abstract class to compute the satck using different partial runs

    Methods
    -------
    (see Stacker in stacking/stacker.py)
    __init__
    __parse_config

    Attributes
    ----------
    (see Stacker in stacking/stacker.py)

    hdu_name: str
    Name of the HDU to containing the spectra to load. Should be a valid HDU in each of the files in stack_list

    stack_list: list of str
    List of files containing the individual stacks to be merged

    stacks: list of (array of float, array of float)
    Individual stacks to be merged. Each item contains a tuple with the flux
    and weight arrays
    


### Class MedianStacker

Class to compute the satck using the median of the different spectra

    Methods
    -------
    (see Stacker in stacking/stacker.py)
    stack

    Attributes
    ----------
    (see Stacker in stacking/stacker.py)

    weighted: boolean
    If True, then compute the weighted median. Otherwise, compute the regular
    median
    


### Class Stacker

Abstract class to define the normalizer skeleton

    Methods
    -------
    __init__
    stack

    Attributes
    ----------
    stacked_flux: array of float
    The stacked flux

    stacked_weight: array of float
    The sum of weights associated with each flux
    


You can now access the options of a given class by executing the following cell

In [4]:
from stacking.configuration_help import print_class_options
print_class_options("MergeSplitMedianStacker", "stackers") # this should be changed with your stacker class

### Class MergeSplitMedianStacker

#### Options:

`hdu name`: Name of the HDU to containing the spectra to load. Should be a valid HDU in each of the files in stack_list. **Type: str**, **Required: no**, **Default: STACK**

`stack list`: List of files containing the individual stacks to be merged. **Type: str**, **Required: yes**

`weighted`: If True, then compute the weighted median. Otherwise, compute the regular median. **Type: bool**, **Required: no**, **Default: False**

## Writer section 
The writer section controls the saving parameters. There are several writing classes that adapt to the different stack types. The class used is automatically picked up by the specified stacker. You can get the current list of available normalizers by running the following cell 

In [5]:
from stacking.configuration_help import print_classes
print_classes("writers")

SplitWriter
BootstrapWriter
StandardWriter
BootstrapSplitWriter


To figure out which writer is being loaded you can run the cell below with your chosen stacker

In [1]:
from stacking.configuration_help import print_selected_writer
print_selected_writer("MergeSplitMedianStacker") # this should be changed with your stacker class

SplitWriter


To get the class description run the cell below

In [2]:
from stacking.configuration_help import print_class_description
print_class_description("SplitWriter", "writers") # this should be changed with your writer class

### Class SplitWriter

Class to write the satck results using splits

    Methods
    -------
    (see Writer in stacking/writer.py)
    __init__
    write_results

    Attributes
    ----------
    (see Writer in stacking/writer.py)

    logger: logging.Logger
    Logger object
    


### Class Writer

Abstract class to write the results

    Methods
    -------
    __init__
    __parse_config
    write_results

    Attributes
    ----------
    output_directory: str
    The output directory
    


You can now access the options of a given class by executing the following cell

In [3]:
from stacking.configuration_help import print_class_options
print_class_options("SplitWriter", "writers") # this should be changed with your writer class

### Class SplitWriter

#### Options:

`output directory`: Directory to save the results. **Type: str**, **Required: yes**

`output file`: Filename to save the results. **Type: str**, **Required: yes**

`overwrite`: Overwrite the output file if it exists. **Type: bool**, **Required: no**

# Examples
You can see some examples of configuration files under the `examples` folder  